# **BDTs at work: the $\mathrm{D}^{+}_\mathrm{s}$ analysis**

The goal of this tutorial is to train and test a multi-class classification algorithm. The tutorial will focus on the reconstruction of the $\mathrm{D^{+}_{s}}$ meson ($\mathrm{c\bar{s}}$) signal in pp collisions with the data collected by ALICE in the first year of Run 3. 

The $\mathrm{D^{+}_{s}}$ meson has quite a short lifetime ($c\tau\sim151\mu\mathrm{m}$), so it cannot be detected directly. Therefore, we need to reconstruct it through its decay products. In this tutorial (as well as in the analyses performed by ALICE) we will reconstruct the $\mathrm{D^{+}_{s}}$ meson through its hadronic decay channel $\mathrm{D^{+}_{s} \rightarrow \mathrm{\phi} \pi^+ \rightarrow K^+ K^- \pi^-}$. _Machine learning_ (ML) algorithms are used to improve the selection of the interesting signal and the rejection of the uninteresting background (arising from the _combination_ of kaons and pions not produced from $\mathrm{D^+_s}$ decays: the _combinatorial background_).

The $\mathrm{D^{+}_{s}}$ mesons can be categorised as either _prompt_ (i.e. originating from charm quark hadronisation) or _non-prompt_ (i.e. originating from beauty-hadron decays). In this tutorial both these classes are considered. In particular, the ML algorithm is going to classify the $\mathrm{D^{+}_{s}}$ meson candidates as either prompt, non-prompt, or background.

Prompt candidates are produced very close to the primary vertex, while non-prompt candidates are produced at a displaced vertex (as beauty hadrons have a lifetime of about c\tau\sim500\mu\mathrm{m}), and therefore we can separate the two classes by looking at the distance between the primary and the D-meson decay vertex. More in general, we will be able to separate signal from background exploiting the _displaced topology_ of the $\mathrm{D^{+}_{s}}$ meson decay.

<img src="img/DsDecaySketch.png" 
     align="center" 
     width="900" />

### **File download**

The first step is to download all the samples we will need for the ML training and test. In Run 3, the tables produced by the analysis workflows in O2Physics can be saved locally as derived data via `TreeCreator` tasks in the form of ROOT `TTrees`. This format is easier to be handled for ML purposes. 

To spare time, the AO2D from hyperloop have been filtered and organised in _.parquet_ files. However, the `TTree` format can be employed as well following the same procedure presented below.

In [ ]:
!curl -L https://cernbox.cern.ch/s/bWelV02GFrfGAcC/download --output data/data_training.parquet
!curl -L https://cernbox.cern.ch/s/ku9JRvffAusJ4Vo/download --output data/data_application.parquet
!curl -L https://cernbox.cern.ch/s/WEeLWC9rlLrfZfJ/download --output data/non_prompt_ds.parquet
!curl -L https://cernbox.cern.ch/s/P98CyjgMdsPwT6O/download --output data/prompt_ds.parquet


## Boosted Decision Trees in a nutshell

That of classifying signal from background is a typical example of _supervised learning_. In a supervised learning task, the objective is to learn to predict or infer a given label or target $y$ based on the
associated features, $x$, assuming that there exists a functional relationship $y = f(x)$ between the two. For our physics analysis, the labels will be prompt, non-prompt, and background. 

The process begins with a training phase, where the model learns (i.e., adjusts its internal parameters) to map the input features $x$ to the corresponding labels $y$ by minimizing a given loss function. In order to train a ML model, a labelled dataset with a well-defined set of features is required. The dataset you are going to use for training your ML model is composed of a number of signal and background examples. To obtain a pure sample of signal candidates, Monte Carlo (MC) simulations are used to generate $\mathrm{D_s^+}$ mesons, while background candidates are obtained from real data.

In particular:
- MC production: _LHC24d3b_ (charm, beauty-enriched)
- Data: _LHC22o_pass7_

In this tutorial, _Boosted Decision Trees_ (BDTs) are employed to classify prompt and non-prompt $\mathrm{D}^{+}_{s}$ candidates, and separate them from the combinatorial background. The building block of a BDT is the _Decision Tree_ algorithm (DT). A decision tree is a structure (similar to a flowchart) where conditions (_nodes_) divide the data into _branches_. "Final" nodes, where the data are not split anymore, are called _leaves_. 

Training a decision tree involves an automated process that recursively constructs the tree using the training set. However, a key limitation of decision trees is their susceptibility to _overfitting_. This means that, if a decision tree is deep enough (where depth is defined as the length of the longest path from the root, i.e., the top node, to a leaf), it can perfectly classify the training set but may struggle to generalise to new data. Overfitting occurs when the model essentially "memorises" the training set rather than learning more general patterns in data. To mitigate this issue, BDT algorithms are employed, which combine multiple shallow trees. In the boosting procedure, decision trees are constructed sequentially, with an emphasis on compensating for misclassified candidates from the previous trees. The resulting model, the BDT, achieves a good performance in terms of both classification accuracy and generalisation to new data.

<img src="img/BDT_sketch.png" 
     align="center" 
     width="900" />

**<font color='red'>_Feedback Time_: are the data been downloaded?**</font>

### **Required python packages**

We first import standard _python packages_ that will be used later on (mainly for plotting)

In [2]:
### standard python library for plotting
import matplotlib.pyplot as plt
plt.ioff()
plt.rcParams["figure.figsize"] = (10, 7)
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

Then, we import the [hipe4ml](https://github.com/hipe4ml/hipe4ml) package.

This package developed in ALICE contains useful methods and classes for dealing with ML analyses.
Two main classes are implemented:
- `TreeHandler`, wrapping uproot and pandas methods: allows for conversion and handling of the training samples (more info about [pandas](https://pandas.pydata.org/) and [uproot](https://uproot.readthedocs.io/en/latest/index.html));
- `ModelHandler`, a common interface for many ML methods;
- `plot_utils`, containing a list of plotting methods to visualize the outcome of the ML analyses.
- `analysis_utils`, containing useful analysis functions

In [ ]:
# import ML-related libraries
import xgboost as xgb # gradient boosting
from hipe4ml.model_handler import ModelHandler
from hipe4ml.tree_handler import TreeHandler
from hipe4ml import plot_utils
from hipe4ml.analysis_utils import train_test_generator

### **Preparing the data set**

We now open the parquet files with the `TreeHandler` class to facilitate the handling of the data for ML purposes.  

In [4]:
# load .parquet files with TreeHandler
hdl_mc_prompt = TreeHandler("data/prompt_ds.parquet")
hdl_mc_nonprompt = TreeHandler("data/non_prompt_ds.parquet")
hdl_data_training = TreeHandler("data/data_training.parquet")

Then, we filter the datasets to select only the 2 < $p_\mathrm{T}$ < 4 GeV/_c_ interval to facilitate and speed up the training process. You can apply such a selection on $p_\mathrm{T}$ (`fPt` variable) using the `apply_preselections` method. 

Typically, the training is performed over several $p_\mathrm{T}$ intervals (e.g. [CERN-EP-2023-162](https://arxiv.org/abs/2308.04877)) to improve the classification, as the decay topology changes quite significantly with $p_\mathrm{T}$ due to the different _Lorentz boost_ of the $\mathrm{D^{+}_{s}}$ meson.

In [5]:
# Preselections (considering just one pt bin)
hdl_mc_prompt.apply_preselections("") # Apply the preselection here

# Remeber to be consistent! Apply the same preselections to the other datasets

As said before, MC simulations are used to obtain a pure sample of signal candidates, while background candidates are obtained from real data. It is usually better to choose "true" data over simulations for the training to avoid eventual shortcomings in the MC, even though it is not always feasible (for example, we cannot use real data for signal examples, as we can't tell signal and background apart).

Typically, since we want to analyse the full dataset, we only use a fraction of the real data for developing our models. This can be done using the `get_subset` method of the `TreeHandler` class. This was already done for you in the parquet files, but you can do it yourself if you want to.

In [ ]:
# we use only a fraction of the data for the training
hdl_data_training = hdl_data_training.get_subset(frac=1.0, rndm_state=42) # rndm_state is the seed for the random number generator. Useful so that we all have the same results

Furthermore, to avoid the inclusion of signal candidates in the background sample, we apply a preselection to the data sample, by selecting only the _sidebands_ of the invariant-mass distribution (`'fM'`) of the candidates. The sideband regions are typically defined as the regions at least 3 $\sigma$ away from the signal peak. In the case of the $\mathrm{D^{+}_{s}}$ meson, you’ll notice that the sidebands are not symmetric around the peak—there’s an interesting reason for this, which will become clear by the end of the tutorial!

In [ ]:
# Apply the preselection here! We only consider the sideband regions in data (M < 1.75 GeV/c^2 or M > 2.1 GeV/c^2)


Another important aspect to consider is the _imbalance_ between the number of signal and background candidates. This imbalance can lead to a bias in the classification, as the algorithm may tend to classify all candidates as the majority class. To mitigate this issue, we can equalize the number of signal and background candidates in the training sample. This can be done using the `get_subset` method.

In [ ]:
print("Number of prompt Ds examples in MC: ", hdl_mc_prompt.get_n_cand())
print("Number of non-prompt Ds examples in MC: ", hdl_mc_nonprompt.get_n_cand())
print("Number of background examples in data: ", hdl_data_training.get_n_cand())
number_cands_to_keep = min(hdl_mc_prompt.get_n_cand(), hdl_mc_nonprompt.get_n_cand(), hdl_data_training.get_n_cand())

You can now try to equalise the number of examples of the hdl_mc_prompt, hdl_mc_nonprompt, and hdl_data_background samples to _number\_cands\_to\_keep_ by using the `get_subset` method. Make sure you assign the result to the same variable to keep the changes.

Hint: you can use the argument `size` of the `get_subset` method to specify the number of examples to keep **in place of** the fraction.

In [8]:
# try it yourself: change the number of candidates to keep
hdl_mc_prompt = 
hdl_mc_nonprompt = 
hdl_data_training = 

Let's check the number of examples in each sample after the equalisation.

In [ ]:
print("Number of prompt Ds examples in MC: ", hdl_mc_prompt.get_n_cand())
print("Number of non-prompt Ds examples in MC: ", hdl_mc_nonprompt.get_n_cand())
print("Number of background examples in data: ", hdl_data_training.get_n_cand())

Let's have a look at the distributions of the features for signal (prompt and non-prompt) and Bkg candidates.
The features can be plotted using the function `plot_utils.plot_distr` of `hipe4ml`.

For simplicity, we select a pool of the possible features. Furthermore, keeping low the number of features in the training will also speed up the application process that will be performed on the Grid! (For this do not miss **Mario**'s session afterwards).  

In [ ]:
hdl_all = [hdl_mc_prompt, hdl_mc_nonprompt, hdl_data_training]
vars_to_draw = ['fCpa', 'fCpaXY', 'fDecayLength', 'fDecayLengthXY', 
                'fDeltaMassPhi', 'fImpactParameterXY', 'fAbsCos3PiK',
                 'fM', 'fPt'] ##just a selection of all the possible variables
leg_labels = ["prompt", "nonprompt", "background"] ## matching the order of the hdl_all

plot_utils.plot_distr(hdl_all, vars_to_draw, bins=100, labels=leg_labels, log=True, density=True, figsize=(12, 7), alpha=0.3, grid=False)
plt.subplots_adjust(left=0.06, bottom=0.06, right=0.99, top=0.96, hspace=0.55, wspace=0.55)
plt.show()

**Q**: Do the distributions look good? 

**Q**: Which features do you expect to have a larger impact on the BDT score? 

Similarly, the correlation matrix for the features is plotted with the function `plt_utils.plot_corr`.

It is important cross-check that none of the features is strongly related to the invariant mass to avoid biasing the measurement!

In [ ]:
plot_utils.plot_corr(hdl_all, vars_to_draw, leg_labels)
plt.show()

**<font color='red'>_Feedback Time_: is everybody's code working?**</font>

A label is assigned to each candidate of the total set: _0_ for prompt $\mathrm{D_s^+}$, _1_ for non-prompt $\mathrm{D_s^+}$, and _2_ for background.

It is important to keep the same order in which the samples are considered also for the labels.

In order to train and test the BDT, the dataset is split into a _train set_ and a _test set_. The train set is used to train the BDT, while the test set is used to evaluate the performance of the trained BDT. It is important to have independent samples for training and testing to avoid biasing the evaluation of the BDT performance.

The fraction of the total set used as test set can be defined through the `test_size` parameter of the function `train_test_generator` and is typically around 0.2 (20%). 

In [11]:
train_test_data = train_test_generator(hdl_all, [0, 1, 2], test_size=0.2, random_state=42)

The ouptut of the function `train_test_generator` is a list containing the training and test sets, as well as the corresponding labels.
Let's split the training set, the test set, and the relative flags. 

In [12]:
train_features = train_test_data[0]
train_labels = train_test_data[1]
test_features = train_test_data[2]
test_labels = train_test_data[3]

### **The model**

For this tutorial, [_XGBoost_](https://xgboost.readthedocs.io/en/latest/) is used as classification algorithm. XGBoost implementation of the gradient boosted decision trees is designed to be highly efficient, flexible and portable. hipe4ml can deal with several implementations other than XGBoost models (e.g. LGBM or sklearn models). However, XGBoost models are usually employed in analyses of the heavy-flavour sector.

The _hipe4ml_ package interfaces with the model through the `ModelHandler` module. The `ModelHandler` is used to define the features used in the training process and to set the _hyperparameters_ of the model, such as the number of estimators, the maximum depth of the trees and the learning rate.

Hyperparameters are parameters that are not learned during the training process, but rather define the model’s architecture and the training process itself. Few of them are listed here below, for a complete description see the [_XGBoost-Doc_](https://xgboost.readthedocs.io/en/stable/parameter.html)
- `max_depth`: Maximum depth of a tree
- `learning_rate`: learning rate of the algorithm. It controls the step size of the gradient descent algorithm
- `n_estimators`: Number of decision trees in the BDT

Let's initialise the algorithm with the features we want to include in the training. **Remember**: exclude the mass and the $p_{\mathrm{T}}$ of the candidate!!! 


In [14]:
features_for_train = [
    'fCpa', 'fCpaXY', 'fDecayLength', 'fDecayLengthXY', 
    'fDeltaMassPhi', 'fImpactParameterXY', 'fAbsCos3PiK'
] 

model_params = {
    'max_depth':5, 'learning_rate':0.029, 'n_estimators':500, 'min_child_weight':2.7,
    'subsample':0.90, 'colsample_bytree':0.97, 'n_jobs':1, 'tree_method': 'hist'
}

model_clf = xgb.XGBClassifier()
model_hdl = ModelHandler(model_clf, features_for_train, model_params)

A huge number of features would lead to a large time and CPU consumption. For this reason, we decided to limit the number of variables in the training.

### **Training and testing the model**
The model training is performed with the method `train_test_model` of the `ModelHandler` module. 
It requires as arguments:
- A list containing: (it's the list we called `train_test_data` before!)
    1. The training set
    2. The true class label for the training set
    3. The test set
    4. The true class label for the test set
- The `multi_class_opt` option

The multi-class option can be set to `ovo` (i.e., _One Vs One_) or `ovr` (i.e., _One Vs Rest_). It is used to define the behaviour of a metric called ROC AUC (_Area Under the Curve_ of the _Receiver Operating Characteristic_), which is used to evaluate the performance of the classifier. You can find more information in the addendum at the end of the tutorial.

In [ ]:
model_hdl.train_test_model(train_test_data, multi_class_opt="ovo")

# the same result can be obtained by splitting the training and test process with the functions below:
# model_hdl.fit(trainset, ytrain, multi_class_opt="ovo")
# model_hdl.predict(testset, ytestset, multi_class_opt="ovo")

**Q:** How do you apply the BDT to the training and test sets?

**Q:** How do you expect the BDT score distributions to be in the training and test sets?

The predictions for the training and the test sets are obtained with the `predict` method which has two arguments:
- the data you want to apply the model to
- `output_margin`, a boolean indicating if we want the raw BDT output (True) or transform it with a [softmax](https://en.wikipedia.org/wiki/Softmax_function) function (False).
The _softmax_ function is used to transform the raw BDT output into a probability distribution (i.e., normalising the scores between 0 and 1).

For this session we use probabilities!

In [16]:
y_pred_train =
y_pred_test =

The results of the training process can be observed by plotting the distributions of the BDT scores for the training and the test sets. The output consists of three scores, related to the probability of belonging to each of the classes (prompt, non-prompt, and background) used for the training. This operation is performed with the method `plot_utils.plot_output_train_test`.

In [ ]:
ml_out_fig = plot_utils.plot_output_train_test(model_hdl, train_test_data, bins=100, 
                                               output_margin=False, labels=leg_labels,
                                               logscale=True, density=True)
plt.show() 

What can we say about the distributions of the BDT scores?
- For each class, the distribution of the BDT scores related to the probability of belonging to that class peaks at high values (i.e., close to 1).
- For each class, the distributions of the BDT scores related to the probability of belonging to the other classes peak at low values (i.e., close to 0).
- The distributions of the model scores obtained from the test set are in good agreement with those obtained from the training set. This is a sign that the model is not significantly affected by overfitting.

We have a good performing model!

### **Feature Importance**

One of the most important (and very interesting) topic in ML is the interpretation of the models output. To understand how a model makes its decisions, the feature importance can be studied. This allows the understanding of which features are more important for the model’s decision-making process, and the optimisation of the feature selection. In addition, the feature importance can be used to check whether the model is learning on the correct features in terms of the physics of the problem. For instance, in the case of D mesons, the most effective features are expected to be the ones associated with the decay length.

The feature importance implemented in the [`SHAP`](https://shap.readthedocs.io/en/latest/index.html) library is used in this hands-on session. An importance value is assigned to each feature, representing the effect on the model prediction of including that feature. The higher the value, the more important the feature is for the model.

**Q**: Which features do you expect to weight more?

The SHAP feature importance can be plotted with the method `plot_utils.plot_feat_imp`.

In [ ]:
shap_figs = plot_utils.plot_feature_imp(test_features, test_labels, model_hdl, labels=leg_labels)  

# only show the SHAP summary plot
plt.close(shap_figs[0])
plt.close(shap_figs[1])
plt.close(shap_figs[2])
plt.show()  

### **Store the trained model**
Once trained and tested, the model can be saved locally.

The model handler from hipe4ml can be saved with the `ModelHandler.dump_model_handler` function. We can then retrieve the model handler for other applications with the `ModelHandler.load_model_handler` function.

The XGBoost model can be saved with the `ModelHandler.dump_original_model` function.

In [19]:
model_hdl.dump_model_handler("./ModelHandler_MulticlassDs_pT_2_4.pickle")
model_hdl.dump_original_model("./XGBoostModel_MulticlassDs_pT_2_4.pickle")

### **Convert the model in ONNX format**
To be applied on the Grid, the XGBoost model has to be converted in ONNX format.

This can be done with the `hipe4ml_converter` package.

In [ ]:
from hipe4ml_converter.h4ml_converter import H4MLConverter

model_converter = H4MLConverter(model_hdl) # create the converter object
model_onnx = model_converter.convert_model_onnx(1)
model_converter.dump_model_onnx("./model_onnx.onnx") # dump the model in ONNX format

**<font color='red'>_Feedback Time_: the models have been saved for everyone?**</font>


### **Apply the model on the dataset we want to analyse!**
Your turn now! Remember that you have downloaded a file called `data_application.parquet` in the `data` folder. This file contains the dataset you want to analyse. You should now load the file with the `TreeHandler` class and apply the model you have just trained on it. You can call the output of the prediction `y_pred`.


In [21]:
hdl_data_appl = 

# apply the model here!
y_pred = 

Now, we will add the BDT score to the dataset we want to analyse, and plot the BDT score distributions for the prompt, non-prompt, and background classes.

You can think the `TreeHandler` class as a wrap around a dataset (more precisely a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html)) that allows to easily manipulate it. We can retrieve the dataset with the `get_data_frame` method and add new columns with the BDT scores with the following code.

In [22]:
df = hdl_data_appl.get_data_frame()
df['prompt_score'] = y_pred[:, 0] # prompt
df['nonprompt_score'] = y_pred[:, 1] # non-prompt
df['background_score'] = y_pred[:, 2] # background

We can now plot the distributions of the BDT scores for the prompt, non-prompt, and background classes.

In [ ]:
df['prompt_score'].plot(kind='hist', bins=100, alpha=0.6, log=True, figsize=(12, 7), grid=False, label='prompt')
df['nonprompt_score'].plot(kind='hist', bins=100, alpha=0.6, log=True, figsize=(12, 7), grid=False, label='nonprompt')
df['background_score'].plot(kind='hist', bins=100, alpha=0.6, log=True, figsize=(12, 7), grid=False, label='background')
plt.xlabel('BDT score')
plt.ylabel('Counts')
plt.legend()
plt.show()

What can we see?

The BDT score distributions for the prompt and non-prompt classes peak at low values, while the BDT score distribution for the background class peaks at high values. We are therefore in presence of a data sample with a lot of background candidates.

(You should consider that this plot shows the distributions of the probabilities of the candidates to belong to the prompt, non-prompt, and background classes, for a **single** dataset. It is different from the similar plot we made before, which showed the distributions of the probabilities of candidates from the **three classes** to belong to **a single** class.)

### **Choose your selections!**
Almost there! We can now apply some selections to the dataset. To do so, you can use the `query` method of pandas.DataFrame, which takes as argument a string containing the selection you want to apply.

Q: What kind of selections would you apply to the dataset?

In [ ]:
selections = 'prompt_score ><= XX and background_score ><= XX' # fill in the XX with the desired values
df_sel = df.query(selections)
hist = df_sel.hist(column='fM', bins=50, range=(1.78, 2.1), figsize=(12, 7), grid=False, density=True)
plt.xlabel(r'$M(\mathrm{KK\pi})$ (GeV/$c^2$)')
plt.ylabel('Counts')
plt.title('')
plt.show()

Congratulations! You have observed a $\mathrm{D^{+}_{s}}$ meson signal!

You can now understand the reason for such a strange sideband definition: we can also recontruct the $\mathrm{D^{+}\rightarrow \mathrm{\phi} \pi^+ \rightarrow K^+ K^- \pi^-}$ meson signal, which is the peak you can see in the left part of the plot.

### Efficiency evaluation

When applying a selection to the dataset, we typically reject a fraction of the signal candidates. Therefore, it is important to evaluate the efficiency of the selection. The efficiency is defined as the fraction of signal candidates that pass the selection.

It is very important, in order not to bias the efficiency evaluation, not to evaluate the efficiency on the same dataset used to train the model. This is because the model has already seen the candidates in the training set and has learned from them. Therefore, the efficiency evaluation should be performed on an independent dataset.

We can use the test set for this purpose. Firstly, let's retrieve the prompt and non-prompt candidates from the test set.

In [25]:
test_features_prompt = test_features[test_labels == 0] # test_labels == 0 is an array of booleans ([True, False, False, ...]), which is used to select the rows of the DataFrame
test_features_nonprompt = test_features[test_labels == 1 ]# test_labels == 1 is an array of booleans ([True, False, False, ...]), which is used to select the rows of the DataFrame

test_scores_prompt = y_pred_test[test_labels == 0]
test_scores_nonprompt = y_pred_test[test_labels == 1]

test_features_prompt['prompt_score'] = test_scores_prompt[:, 0] # prompt
test_features_prompt['nonprompt_score'] = test_scores_prompt[:, 1] # non-prompt
test_features_prompt['background_score'] = test_scores_prompt[:, 2] # background

test_features_nonprompt['prompt_score'] = test_scores_nonprompt[:, 0] # prompt
test_features_nonprompt['nonprompt_score'] = test_scores_nonprompt[:, 1] # non-prompt
test_features_nonprompt['background_score'] = test_scores_nonprompt[:, 2] # background

Now, we can apply the selection to the test set and evaluate the efficiency. You can try by yourself to apply the selection to the test set and evaluate the efficiency.

Keep in mind that `test_features_prompt` and `test_features_nonprompt` are `pandas.DataFrame`s (which you used before).

You can retrieve the number of entries in a `pandas.DataFrame` with the `len` function.

In [ ]:
sel_df_prompt = test_features_prompt.query(selections)
sel_df_nonprompt = test_features_nonprompt.query(selections)

# Your turn! You can evaluate the number of entries of the pandas.DataFrame named df using len(df)
eff_prompt =
eff_nonprompt =

print("Prompt efficiency: ", eff_prompt)
print("Nonprompt efficiency: ", eff_nonprompt)

We can also study how the efficiency changes with the selection on the prompt BDT score. We can plot the efficiency as a function of the BDT score.

In [ ]:
prompt_scores = np.linspace(0.0, 0.85, 86) # [0.00, 0.01, 0.02, ..., 0.85]
efficiencies_prompt = [] # list to store the efficiencies (similar to an std::vector in C++)
efficiencies_non_prompt = [] # list to store the efficiencies (similar to an std::vector in C++)

# loop over the prompt_scores
for score in prompt_scores:
    selections = f'prompt_score > {score} and background_score < 0.05'
    sel_df_prompt = test_features_prompt.query(selections)
    efficiencies_prompt.append(len(sel_df_prompt) / len(test_features_prompt)) # similar to push_back in C++
    sel_df_non_prompt = test_features_nonprompt.query(selections)
    efficiencies_non_prompt.append(len(sel_df_non_prompt) / len(test_features_nonprompt)) # similar to push_back in C++

plt.plot(prompt_scores, efficiencies_prompt, label='prompt', marker='o')
plt.plot(prompt_scores, efficiencies_non_prompt, label='nonprompt', marker='o')
plt.xlabel('Prompt score >')
plt.ylabel('Efficiency')
plt.legend()
plt.show()

As you can see, the selection on the prompt score does a very good job in rejecting non-prompt candidates. The efficiency is very high for prompt candidates, while it is very low for non-prompt candidates.

At very high threshold values, the efficiency for prompt candidates steeply decreases. For your future analyses, a key point will be to find a working point providing the best compromise between signal efficiency and background rejection. 

# **BEFORE THE COFFEE BREAK!!!** 

Don't forget to download the AO2D file necessary for the next step of the tutorial! You can do this (**on your laptop**) as
```bash
curl -L https://cernbox.cern.ch/s/uIR0aumtfT14NIm/download --output AO2D_MC_Ds.root
```
It will take some time to complete.

# **Addenda**

### **Optimisation of hyperparameters with Optuna**

The optimisation of the hyperparameters is a key step to obtain the best performance from the algorithm. In _hipe4ml_ the [`Optuna`](https://github.com/optuna/optuna) package is employed for the optimisation through the method `ModelHandler.optimize_params_optuna`.

The difference between Optuna and other packages dedicated to hyperparameters optimisation based on grid search or random search is that Optuna takes into account the past hyperparameter configuration evaluations when choosing the configuration to evaluate next. (More info about Optuna can be found [here](https://arxiv.org/abs/1907.10902)).

A set of hyperparameters should be tested on different sample than the one used for the training and testing, called _validation sample_, to avoid overfitting problems. Since the number of events is limited, an approach called _k-fold cross validation_ is used. 

In the cross validation procedure, the original sample is divided in _k_ parts called _folds_. For each set of hyperparameters, _n-1_ folds are used for the optimisation and the remaining one as validation. This operation is repeated after permuting the folds used for optimisation and for testing and the final result is the mean value of all the permutations. The hyperparameters that give the best performance are then used for the training.

The _ModelHandler_ automatically updates the hyperparameters after their optimisation.

To spare time, in this tutorial the hyperparameter optimisation and the number of configuration was limited.

The optimised configuration can be saved and loaded afterwards. Let's have a look!

<img src="img/k_fold.png" 
     align="center" 
     width="900" />

You can try to optimise the BDT yourself with Optuna with the following commands.

Remember to set the range in which you want to optimise the hyperparameters, the metric you want to optimise, if you want to maximise or minimise it, and the number of trials you want to try.

In [ ]:
import optuna as op
import pickle

hyper_pars_ranges = {"max_depth": (2, 4), "learning_rate": (0.01, 0.1)}
opstudy = model_hdl.optimize_params_optuna(
    train_test_data, hyper_pars_ranges, cross_val_scoring="roc_auc_ovo", n_jobs=8,
    n_trials=10, direction="maximize", save_study='optuna_study.pkl'
)
fig = op.visualization.matplotlib.plot_optimization_history(opstudy) 
fig = op.visualization.matplotlib.plot_param_importances(opstudy)
fig = op.visualization.matplotlib.plot_contour(opstudy)
plt.show()

### **Receiver Operating Curve**
The quality of the algorithm can be also studied with the _Receiver Operating Characteristic_ (_ROC_) curve. The ROC curve which represents the true positive rate (TPR) against the false positive rate (FPR) for different thresholds on the BDT ouptut values. TPR and FPR are defined as:

$\mathrm{TPR}=\frac{\sum \mathrm{TP}}{\sum \mathrm{TP} + \sum \mathrm{FN}} \hspace{2cm} \mathrm{FPR}=\frac{\sum \mathrm{FP}}{\sum \mathrm{FP} + \sum \mathrm{TN}} $

The true positives (TP) are the number of correctly classified positive candidates, the false positives (FP) are the number of negative candidates being mistakenly classified as positives, and the true negatives (TN) and false negatives (FN) are defined similarly.

In the case of multi-classifiaction problems, the ROC curve is not natively defined. Hence, we can generalise it by either employing the _One Vs One_ or the _One Vs Rest_ approach. In this tutorial, we adopt the _One Vs One_, which is a plot of the TPR against the FPR for a given pair of classes. For a classification problem with $N$ classes, there are $N(N − 1)$ possible pairs of ordered classes, and therefore of _One Vs One_ ROC curves.

The most common way employed to evaluate the performance of a BDT is to compute the _Area Under the Curve_ ROC, called AUC. The ROC AUC ranges from 0 to 1. A random classifier has a ROC AUC of 0.5, while a perfect classifier has a ROC AUC of 1.

A good model classifier is characterised by a large area under the ROC curve (_ROC AUC_).

**Q**: How do you expect the ROC Curve to be?

**Q**: How do you expect the ROC AUC to be?

**Q**: Do you expect any difference between training and test? 


The ROC curve can be plotted with the method `plot_utils.plot_roc_train_test`. 

In [ ]:
# this will produce 3 plots: the ROC curves for the training and test sets, and the a comparison of the two
roc_train_test_fig = plot_utils.plot_roc_train_test(train_test_data[3], y_pred_test, train_test_data[1], y_pred_train, None, leg_labels, multi_class_opt="ovo")

plt.show()

For a binary classification task (only 2 classes involved) the output of the ML model is a single value ranging from 0 to 1, and can be interpreted as the probability of the candidate belonging to the positive class. If positive candidates are selected as those with a score greater than a certain threshold $t$, when $t = 0$ all candidates are classified as positive, and both the TPR and FPR will be equal to 1. On the other hand, if $t = 1$, no candidate is classified as positive, and the TPR and FPR will both equal 0. Therefore, the different values of $t$ between 0 and 1, which yield different TPR and FPR values, will trace the ROC curve, going from the point (1,1) to (0,0) as $t$ increases.

Evaluating the ROC AUC in training and test a slight difference is found. The higher ROC AUC in training compared to test is a systematic behaviour due to the small presence of overfitting, which indicates a successful training.

### **References**
- [Deep Learning and Its Application to LHC Physics](https://arxiv.org/pdf/1806.11484.pdf)
- [Machine learning at the energy and intensity frontiers of particle physics](https://www.nature.com/articles/s41586-018-0361-2)
- [HEP ML Living Review](https://github.com/iml-wg/HEPML-LivingReview)
- [Inter-Experimental LHC Machine Learning Working Group](https://github.com/iml-wg)

Remember to follow the [ALICE ML meetings](https://indico.cern.ch/category/7570/) for many more inputs and applications of ML techniques!
Contact [Francesco](francesco.mazzaschi@cern.ch) and [Fabio](fabio.catalano@cern.ch) for any questions ;)